In [1]:
import pandas as pd
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding


## Load data

In [2]:
cerebras_df = pd.read_excel("data/testi_cerebras_20250608.xlsx")
mistral_df = pd.read_excel("data/testi_mistral_20250608.xlsx")
qwen_df = pd.read_excel("data/testi_qwen_20250608.xlsx")

In [3]:
cerebras_df["model"] = "llama-4-scout-17b-16e-instruct"
mistral_df["model"] = "mistral-large-latest"
qwen_df["model"] = "qwen2.5-72b-instruct"

## Merge data and random sample for training

In [14]:
all_df = pd.concat([cerebras_df, mistral_df, qwen_df], ignore_index=True)
all_df["text_id"] = all_df.testo.astype("category").cat.codes


train_df = all_df.sample(frac=1, random_state=42).drop_duplicates(subset='testo', keep='first').reset_index(drop=True)

In [46]:
all_df.to_excel("data/testi_all.xlsx", index=False)

In [15]:
train_df.head()

,edizione,sezione,titoletto,testo,input,model,text_id
0,2023Q2,corpo,L’occupazione continuerebbe a espandersi,Il saldo fra la quota di aziende che intendono...,Il saldo tra le aziende che vogliono aumentare...,qwen2.5-72b-instruct,33
1,2023Q3,corpo,Le aspettative di inflazione sono scese su tut...,Le aspettative sull’inflazione al consumo si s...,Le aspettative sull'inflazione al consumo si s...,qwen2.5-72b-instruct,84
2,2020Q3,sintesi,NaN,Secondo l’indagine condotta tra il 27 agosto e...,Ecco una possibile bozza precedente:\n\nDall'i...,llama-4-scout-17b-16e-instruct,251
3,2024Q3,corpo,Le aspettative di inflazione sono lievemente a...,"Rispetto alla scorsa rilevazione, l’inflazione...","Okay, quindi, rispetto all'ultima rilevazione,...",mistral-large-latest,227
4,2021Q2,corpo,…così come quelli sull’andamento corrente e at...,Le imprese si attendono anche una forte cresci...,Le imprese pensano che la domanda dei loro pro...,llama-4-scout-17b-16e-instruct,135


In [6]:
train_df.model.value_counts()

model
qwen2.5-72b-instruct              93
llama-4-scout-17b-16e-instruct    87
mistral-large-latest              83
Name: count, dtype: int64

## Add one-shot example for prompting in training

### Qdrant collection to select the most simliar - non identical example

In [18]:
client = QdrantClient(":memory:")  # Qdrant is running from RAM.
emb = TextEmbedding("jinaai/jina-embeddings-v3")

_ = client.create_collection(
    collection_name="examples",
    vectors_config=models.VectorParams(
        size=emb.model.model_description.dim, 
        distance=models.Distance.COSINE
    ),  # size and distance are model dependent
)

In [19]:
train_dicts = train_df.to_dict(orient="records")
all_dicts = all_df.to_dict(orient="records")

In [20]:
_ = client.upload_collection(
    collection_name="examples",
    vectors=[emb.embed(doc["testo"]).send(None) for doc in all_dicts],
    payload=all_dicts,
    ids=range(len(all_dicts)),
)

In [42]:
results = []

for row in train_dicts:
    # Find the most simliar text - not identical to the current one
    os_dict = client.query_points(
        collection_name="examples",
        query=emb.embed(row["testo"]).send(None),
        limit=1,
        query_filter=models.Filter(
            must_not=[
                models.FieldCondition(
                    key="text_id",
                    match=models.MatchValue(
                        value=row["text_id"],
                    ),
                )
            ]
        ),
    ).points[0].payload
    new_row = {
        **row,
        "os_testo": os_dict["testo"],
        "os_input": os_dict["input"],
        "os_model": os_dict["model"],
        "os_text_id": os_dict["text_id"],
    }
    results.append(new_row)


In [43]:
train_os_df = pd.DataFrame(results)

In [44]:
train_os_df.head()

,edizione,sezione,titoletto,testo,input,model,text_id,os_testo,os_input,os_model,os_text_id
0,2023Q2,corpo,L’occupazione continuerebbe a espandersi,Il saldo fra la quota di aziende che intendono...,Il saldo tra le aziende che vogliono aumentare...,qwen2.5-72b-instruct,33,ll divario tra le quote di aziende che prevedo...,Il divario tra le aziende che prevedono di aum...,qwen2.5-72b-instruct,261
1,2023Q3,corpo,Le aspettative di inflazione sono scese su tut...,Le aspettative sull’inflazione al consumo si s...,Le aspettative sull'inflazione al consumo si s...,qwen2.5-72b-instruct,84,Le aspettative sull’inflazione al consumo si s...,Le aspettative sull'inflazione al consumo si s...,qwen2.5-72b-instruct,83
2,2020Q3,sintesi,NaN,Secondo l’indagine condotta tra il 27 agosto e...,Ecco una possibile bozza precedente:\n\nDall'i...,llama-4-scout-17b-16e-instruct,251,Secondo l’indagine condotta tra il 23 novembre...,Abbiamo fatto un'indagine tra il 23 novembre e...,llama-4-scout-17b-16e-instruct,242
3,2024Q3,corpo,Le aspettative di inflazione sono lievemente a...,"Rispetto alla scorsa rilevazione, l’inflazione...","Okay, quindi, rispetto all'ultima rilevazione,...",mistral-large-latest,227,L’inflazione al consumo sui 12 mesi attesa dal...,"L'inflazione al consumo sui 12 mesi, quella ch...",mistral-large-latest,161
4,2021Q2,corpo,…così come quelli sull’andamento corrente e at...,Le imprese si attendono anche una forte cresci...,Le imprese pensano che la domanda dei loro pro...,llama-4-scout-17b-16e-instruct,135,La differenza tra la quota di imprese che hann...,"Allora, la differenza fra la quota di imprese ...",mistral-large-latest,58


In [45]:
train_os_df.to_excel("data/testi_randomized.xlsx", index=False)